[View in Colaboratory](https://colab.research.google.com/github/rehabreda/machine_learning/blob/master/sentiment.ipynb)

In [0]:

import nltk
import pickle 
import random
import numpy as np
import tensorflow as tf
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [0]:
lemmatizier=WordNetLemmatizer()
hm_lines=100000

In [0]:
def create_lexicon(pos,neg):
  lexicon=[]
  with open(pos,'r',encoding="utf-8",errors = 'ignore') as f:
    contents=f.readlines()
    for line in contents[:hm_lines]:
      words=word_tokenize(line)
      lexicon+=list(words)
      
  with open(neg,'r',encoding="utf-8",errors = 'ignore') as f:
    contents=f.readlines()
    for line in contents[:hm_lines]:
      words=word_tokenize(line)
      lexicon+=list(words)  
      
      
  lexicon=[lemmatizier.lemmatize(i) for i in lexicon]
  word_counts=Counter(lexicon)
  l2=[]
  for w in word_counts:
    if 1000>word_counts[w]>50:
      l2.append(w)
      
      
  return l2    

In [0]:
def sample_handling(sample,lexicon,classification):
  featureset=[]
  with open(sample,'r',encoding="utf-8",errors = 'ignore') as f:
    contents=f.readlines()
    for line in contents[:hm_lines]:
      current_words=word_tokenize(line.lower())
      current_word=[lemmatizier.lemmatize(i) for i in current_words]
      features=np.zeros(len(lexicon))
      for word in current_words:
        if word.lower() in lexicon:
          index=lexicon.index(word.lower())
          features[index]=+1
          
      features=list(features)
      featureset.append([features,classification])
      return featureset
      

In [0]:
def create_feature_sets_and_labels(pos,neg,test_size = 0.1):
  lexicon=create_lexicon(pos,neg)
  features=[]
  features+=sample_handling('pos.txt',lexicon,[1,0])
  features+=sample_handling('neg.txt',lexicon,[0,1])
  random.shuffle(features)
  features=np.array(features)
  testing_size=int(test_size*len(features))
  traing_x=list(features[:,0][:-testing_size])
  traing_y=list(features[:,1][:-testing_size])
  testing_x=list(features[:,0][-testing_size:])
  testing_y=list(features[:,1][-testing_size:])
  return train_x,train_y,test_x,test_y

In [61]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving pos.txt to pos.txt
User uploaded file "pos.txt" with length 626349 bytes


In [62]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  

Saving neg.txt to neg.txt
User uploaded file "neg.txt" with length 612459 bytes


In [0]:
train_x,train_y,test_x,test_y=create_feature_sets_and_labels('pos.txt','neg.txt')

In [0]:
n_nodes_hl1 = 1500
n_nodes_hl2 = 1500
n_nodes_hl3 = 1500

n_classes = 2
batch_size = 100

x = tf.placeholder('float')
y = tf.placeholder('float')

def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([len(train_x[0]), n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}


    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            i=0
            while i<len(train_x):
              start=i
              end=i+batch_size
              
              epoch_x=np.array(train_x[start:end])
              epoch_y=np.array(train_y[start:end])
              
              _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
              epoch_loss += c
              i+=batch_size

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))


In [96]:
train_neural_network(x)

Epoch 0 completed out of 10 loss: 1168874.0151367188
Epoch 1 completed out of 10 loss: 437075.97814941406
Epoch 2 completed out of 10 loss: 221016.7059326172
Epoch 3 completed out of 10 loss: 212563.4541015625
Epoch 4 completed out of 10 loss: 366917.1104736328
Epoch 5 completed out of 10 loss: 139664.1460571289
Epoch 6 completed out of 10 loss: 104142.62270355225
Epoch 7 completed out of 10 loss: 80649.97135925293
Epoch 8 completed out of 10 loss: 25258.51383972168
Epoch 9 completed out of 10 loss: 9386.23272895813
Accuracy: 0.63977486
